In [1]:
from torch import nn
from torchvision import transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

from ml_tools.ML_vision_datasetmaster import DragonDatasetVision
from ml_tools.ML_trainer import DragonTrainer
from ml_tools.ML_callbacks import DragonModelCheckpoint, DragonEarlyStopping, DragonLRScheduler
from ml_tools.ML_utilities import inspect_model_architecture
from ml_tools.ML_configuration import MultiClassImageClassificationMetricsFormat, FinalizeMultiClassImageClassification
from ml_tools import custom_logger

from rootpaths import PM
from visual_ccc.gradcam import custom_alexnet, SIZE_REQUIREMENT

## Multiclass classification: Dendrites, Spheroids, Alloys

In [2]:
VAL_SIZE = 0.2
TEST_SIZE = 0.1
RANDOM_STATE = 101

vision_dataset = DragonDatasetVision.from_folder(PM.original_images)

vision_dataset.split_data(val_size=VAL_SIZE, 
                          test_size=TEST_SIZE,
                          random_state=RANDOM_STATE)

vision_dataset.configure_transforms(resize_size=int(1.2*SIZE_REQUIREMENT),
                                    crop_size=SIZE_REQUIREMENT,
                                    mean=None, std=None,
                                    pre_transforms=[transforms.Grayscale(num_output_channels=1)])

train_dataset, validation_dataset, test_dataset = vision_dataset.get_datasets()


🐉 2025-11-10 12:38 [✅ INFO] - Found 112 images in 3 classes.

🐉 2025-11-10 12:38 [✅ INFO] - Test set created with 12 images.

🐉 2025-11-10 12:38 [✅ INFO] - Data split into: 
- Training: 78 images 
- Validation: 22 images

🐉 2025-11-10 12:38 [✅ INFO] - Image transforms configured and applied.


In [25]:
class_map = vision_dataset.save_class_map(save_dir=PM.artifacts)

vision_dataset.save_transform_recipe(filepath=PM.transform_recipe_file)


🐉 2025-11-10 13:09 [✅ INFO] - Log saved as: 'Class_to_Index'

🐉 2025-11-10 13:09 [✅ INFO] - Transform recipe saved as 'transform_recipe.json'.


In [26]:
INITIAL_LR = 0.0002
SCHEDULER_PATIENCE = 2
# STOP_PATIENCE = 12

# Model
model = custom_alexnet()
original_in_features: int = model.classifier[6].in_features # type: ignore
model.classifier[6] = nn.Linear(in_features=original_in_features, out_features=3, bias=True) # 3 class classification

inspect_model_architecture(model=model, save_dir=PM.artifacts)

# Optimizer
optimizer = AdamW(params=model.parameters(), lr=INITIAL_LR)

# Trainer
trainer = DragonTrainer(model=model,
                    train_dataset=train_dataset,
                    validation_dataset=validation_dataset,
                    kind="multiclass image classification",
                    optimizer=optimizer,
                    criterion="auto",
                    device="cuda:0",
                    checkpoint_callback=DragonModelCheckpoint(save_dir=PM.checkpoints, mode="min"),
                    early_stopping_callback=None,
                    # early_stopping_callback=DragonEarlyStopping(patience=STOP_PATIENCE, mode="min"),
                    lr_scheduler_callback=DragonLRScheduler(scheduler=ReduceLROnPlateau(optimizer=optimizer, 
                                                                                        mode="min",
                                                                                        factor=0.7,
                                                                                        patience=SCHEDULER_PATIENCE))
                    )


🐉 2025-11-10 13:09 [✅ INFO] - Model architecture summary saved to 'model_architecture_summary.txt'


In [27]:
BATCH_SIZE = 2

history = trainer.fit(save_dir=PM.artifacts, epochs=100, batch_size=BATCH_SIZE)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/100:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch 2/100:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch 3/100:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch 4/100:   0%|          | 0/39 [00:00<?, ?it/s]

...


🐉 2025-11-10 13:13 [✅ INFO] - Epoch 97: Learning rate changed to 0.000000


Epoch 98/100:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch 99/100:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch 100/100:   0%|          | 0/39 [00:00<?, ?it/s]


🐉 2025-11-10 13:13 [✅ INFO] - 📉 Loss plot saved as 'loss_plot.svg'


In [28]:
train_log = {
    "validation size": VAL_SIZE,
    "test size": TEST_SIZE,
    "images per dataset": vision_dataset.images_per_dataset(),
    "random state": RANDOM_STATE,
    "initial lr": INITIAL_LR,
    "scheduler patience": SCHEDULER_PATIENCE,
    # "stop patience": STOP_PATIENCE,
    "batch size": BATCH_SIZE,
    "history": history
}

custom_logger(data=train_log,
              save_directory=PM.results,
              log_name="train_log",
              dict_as="json")


🐉 2025-11-10 13:13 [✅ INFO] - Log saved as: 'train_log_20251110_131318'


In [32]:
trainer.evaluate(
    save_dir=PM.metrics, 
    model_checkpoint="latest",
    classification_threshold=0.60,
    test_data=test_dataset,
    val_format_configuration=MultiClassImageClassificationMetricsFormat(cmap='BuGn',
                                                                    ROC_PR_line="darkorange"),
    test_format_configuration=MultiClassImageClassificationMetricsFormat(cmap='BuPu',
                                                                     ROC_PR_line="forestgreen")
)


🐉 2025-11-10 13:21 [✅ INFO] - Evaluating on validation dataset. Metrics will be saved to 'Validation_Metrics'

🐉 2025-11-10 13:21 [✅ INFO] - Loading checkpoint from 'epoch37_PyModelCheckpoint_0.3277.pth'...

🐉 2025-11-10 13:21 [✅ INFO] - Restored training history up to epoch 37.

🐉 2025-11-10 13:21 [✅ INFO] - Restored LR scheduler state for: ReduceLROnPlateau

🐉 2025-11-10 13:21 [✅ INFO] - Restored DragonModelCheckpoint 'best' score to: 0.3277

🐉 2025-11-10 13:21 [✅ INFO] - Checkpoint loaded. Resuming training from epoch 38.

🐉 2025-11-10 13:22 [✅ INFO] - 📝 Classification report saved as 'classification_report.txt'

🐉 2025-11-10 13:22 [✅ INFO] - 📊 Report heatmap saved as 'classification_report_heatmap.svg'

🐉 2025-11-10 13:22 [✅ INFO] - ❇️ Confusion matrix saved as 'confusion_matrix.svg'

🐉 2025-11-10 13:22 [✅ INFO] - 💾 Optimal threshold saved as 'best_threshold_Alloys.txt'

🐉 2025-11-10 13:22 [✅ INFO] - 💾 Optimal threshold saved as 'best_threshold_Dendrites.txt'

🐉 2025-11-10 13:22 [

In [ ]:
finalizer = FinalizeMultiClassImageClassification(filename="DendritesSpheroids", class_map=class_map)

trainer.finalize_model_training(model_checkpoint="current",
                                save_dir=PM.artifacts,
                                finalize_config=finalizer)